In [ ]:
'''
# Goal of this tutorial is::::: to write a code in in Pytorch to classify the digits from 0 to 9

# Input: a list containing one digit numbers [0 to 9]
# Process: 
      1. Model will learn and classify every element of the input array
      2. We compute the loss by comparing the predited and input arrays
      3. Backtrack the loss for model learning

# Output: The array, presenting the output of the model => classification of the input array

# Example:
    input:  [3,4,5,6,7,8,9,1,2,0]
    output: [3,4,2,2,7,8,9,1,2,0]

    Explaination: The network classified 5 and 6 as 2 which is incorrect. Accuracy is 8/10.

Mohammad Zohaib
PhD Student
PAVIS, IIT Genoa Itlay
engr.mz@hotmail.com
Last update: August 21, 2022

'''

In [ ]:
# Import the libraries

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter   # Plot the loss functions


In [ ]:
# Define Hyperparameters

input_size = 1 # list of 10 number 
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 50 # number of times which the entire dataset is passed throughout the model
batch_size = 1024 # the size of input data took for one iteration
data_set_size = 100000 # size of the train dataset [total lists of digits] => validation set = 30% of the train set
lr = 1e-3 # size of step 

In [ ]:
# Prepare lists of the data [input, labels(ground truths)]


train_data = torch.randint(0,9, (data_set_size,1))
train_labels = torch.zeros(data_set_size, num_classes)
for i in range(len(train_data)):
  train_labels[i][train_data[i]]=1

train_set = []
for i in range(len(train_data)):
  train_set.append([train_data[i], train_labels[i]])


validation_data = torch.randint(0,9, (int(data_set_size*0.2),1))
validation_labels = torch.zeros(int(data_set_size*0.2), num_classes)
for i in range(len(validation_data)):
  validation_labels[i][validation_data[i]]=1


validation_set = []
for i in range(len(validation_data)):
  validation_set.append([validation_data[i], validation_labels[i]])

print("Loaded Examples => train: {}, test: {}".format(len(train_set), len(validation_data)))

Loaded Examples => train: 100000, test: 20000


In [ ]:
# Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_set,
                                             batch_size = batch_size,
                                             shuffle = True, drop_last=False)

validation_gen = torch.utils.data.DataLoader(dataset = validation_set,
                                      batch_size = batch_size, 
                                      shuffle = False, drop_last=False)

In [ ]:
# Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.bn = nn.BatchNorm1d(hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
    # self.softmax = nn.Softmax(dim=1)   

  def forward(self,x):
    x = self.fc1(x)
    x = self.bn(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [ ]:
# Build the model

model = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  model.cuda()

In [ ]:
# Define loss-function & optimizer

# loss_function = nn.L1Loss()
# loss_function = nn.MSELoss()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training the model

writer = SummaryWriter('train_summary')

best_loss = 1e-10
best_epoch = 0

for epoch in range(num_epochs):

  model.train()
  train_loss = 0
  for i ,(data,labels) in enumerate(train_gen):
    data = data.float().cuda()
    labels = labels.float().cuda()
    predictions = model(data)
    loss = loss_function(predictions, labels) 
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  train_loss /= len(train_data)/batch_size


  model.eval()
  val_loss = 0
  for i ,(data,labels) in enumerate(validation_gen):
    data = data.float().cuda()
    labels = labels.float().cuda()
    predictions = model(data)
    loss = loss_function(predictions, labels)
    val_loss += loss.item()
  val_loss /= len(validation_data)/batch_size

  if best_loss < val_loss:
    best_epoch = epoch

  if (epoch+1) % 10 == 0:
    print('Epoch [{}/{}], Loss: train: {}, validation: {}'.format(epoch+1, num_epochs, train_loss, val_loss))
  writer.add_scalars('Loss', {'training':train_loss, 'validation_loss':val_loss}, epoch)

print('Best epoch {}, Loss: {}'.format(best_epoch, best_loss))
writer.close()

Epoch [10/50], Loss: train: 0.594971176147461, validation: 0.492543571472168
Epoch [20/50], Loss: train: 0.35697651485443116, validation: 0.5499215637207031
Epoch [30/50], Loss: train: 0.044309474172592164, validation: 0.05298171138763428
Epoch [40/50], Loss: train: 0.008136947325468063, validation: 3.078944571316242e-05
Epoch [50/50], Loss: train: 0.006441427914144006, validation: 2.9371616430580615e-06
Best epoch 49, Loss: 1e-10


In [ ]:
# Display the loss functions

%load_ext tensorboard
%tensorboard --logdir train_summary

In [ ]:
# Test the trained model for limited test samples

test_input = torch.randint(0,9, (30, 1)).float().cuda()
model.eval()
predictions = model(test_input)
# print(predictions)
predictions = torch.argmax(predictions, dim=1).to('cpu').float().numpy()
test_input = test_input.squeeze().to('cpu').numpy()
print("Test data:  {} \nPrediction: {}".format(test_input, predictions))
print("Accuracy is: {:.2f}".format(100*(predictions == test_input).sum()/len(test_input)))


Test data:  [8. 2. 0. 8. 2. 5. 5. 4. 2. 4. 4. 5. 3. 3. 8. 5. 5. 4. 0. 7. 1. 4. 4. 4.
 5. 5. 6. 1. 0. 1.] 
Prediction: [8. 2. 0. 8. 2. 5. 5. 4. 2. 4. 4. 5. 3. 3. 8. 5. 5. 4. 0. 7. 1. 4. 4. 4.
 5. 5. 6. 1. 0. 1.]
Accuracy is: 100.00


In [ ]:
# Test the model for whole test set

''' Creating the test dataset => 30% of the train dataset '''
test_data = torch.randint(0,9, (int(data_set_size*0.3),1))
test_labels = torch.zeros(int(data_set_size*0.3), num_classes)
for i in range(len(test_data)):
  test_labels[i][test_data[i]]=1

test_set = []
for i in range(len(test_data)):
  test_set.append([test_data[i], test_labels[i]])


''' Creating the dataloading pipeline'''
test_gen = torch.utils.data.DataLoader(dataset = test_set,
                                      batch_size = batch_size, 
                                      shuffle = False, drop_last=False)



''' Testing the model => eval() mode'''
model.eval()
test_loss = 0
test_accuracy = 0
for i ,(data,labels) in enumerate(test_gen):
    data = data.float().cuda()
    labels = labels.float().cuda()
    predictions = model(data)
    loss = loss_function(predictions, labels)
    test_loss += loss.item()

    predictions = torch.argmax(predictions, dim=1).to('cpu').float().numpy()
    test_input = data.squeeze().to('cpu').numpy()
    test_accuracy += (predictions == test_input).sum()


test_loss /= len(test_data)/batch_size
test_accuracy = test_accuracy*100/len(test_data)

print("Loss: {:.2f},  Accuracy: {:.2f}".format(test_loss, test_accuracy))
print()


Loss: 0.00,  Accuracy: 100.00

